# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_data = pd.read_csv("Data_CSV/city_weather_data.csv", encoding="ISO-8859-1")
city_weather_data.head()


,Unnamed: 0,City,Max Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,0,Vaini,78.80,94,75,14.97,-21.2000,-175.2000
1,1,Khatanga,-29.72,79,0,10.65,71.9667,102.5000
2,2,Talcahuano,62.60,82,75,6.91,-36.7167,-73.1167
3,3,Tasiilaq,26.60,46,0,18.41,65.6145,-37.6368
4,4,SÄmbhar,53.60,62,0,3.20,26.9167,75.2000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = city_weather_data[['Latitude', 'Longitude']]
weight = city_weather_data['Humidity']

In [4]:
layout = {'width': '900px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

figure = gmaps.figure(layout = layout)

figure = gmaps.figure(center=(40.0, -5.0), zoom_level=2) 
max_intensity = np.max(weight)

heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating=False, max_intensity=100, point_radius=3)

figure.add_layer(heat_layer)

figure


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
my_vacation_cities = city_weather_data[(city_weather_data['Max Temperature']>60)&(city_weather_data['Max Temperature']<90)
                                       &(city_weather_data['Cloudiness']<30)&(city_weather_data['Humidity']<70)]
my_vacation_cities.head(20)

,Unnamed: 0,City,Max Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
6,6,Rafaela,89.01,27,0,1.01,-31.2503,-61.4867
11,11,Punta Arenas,71.60,24,0,13.80,-53.1500,-70.9167
20,20,Aden,75.42,67,0,5.44,12.7794,45.0367
38,38,Palauig,77.09,69,13,4.50,15.4350,119.9045
45,45,Gwadar,65.43,56,0,6.64,25.1216,62.3254
47,47,Marzuq,60.55,59,0,4.09,14.4000,46.4667
48,48,Cidreira,72.41,69,0,8.93,-30.1811,-50.2056
52,52,Kapaa,75.99,56,1,5.75,22.0752,-159.3190
70,70,Cabo San Lucas,75.00,53,20,11.50,22.8909,-109.9124
74,74,Nava,77.00,44,1,16.11,28.4167,-100.7667


In [6]:
my_vacation_cities.reset_index(drop=True, inplace=True)
my_vacation_cities


,Unnamed: 0,City,Max Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,6,Rafaela,89.01,27,0,1.01,-31.2503,-61.4867
1,11,Punta Arenas,71.60,24,0,13.80,-53.1500,-70.9167
2,20,Aden,75.42,67,0,5.44,12.7794,45.0367
3,38,Palauig,77.09,69,13,4.50,15.4350,119.9045
4,45,Gwadar,65.43,56,0,6.64,25.1216,62.3254
...,...,...,...,...,...,...,...,...
71,569,El Alto,76.57,69,8,15.75,-4.2714,-81.2147
72,571,Laguna,66.00,42,1,4.36,38.4210,-121.4238
73,573,Ixtapa,75.20,69,1,13.80,20.7000,-105.2000
74,574,San Vicente,74.95,49,0,4.99,-26.6167,-54.1333


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame(my_vacation_cities)
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,City,Max Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Hotel Name
0,6,Rafaela,89.01,27,0,1.01,-31.2503,-61.4867,
1,11,Punta Arenas,71.60,24,0,13.80,-53.1500,-70.9167,
2,20,Aden,75.42,67,0,5.44,12.7794,45.0367,
3,38,Palauig,77.09,69,13,4.50,15.4350,119.9045,
4,45,Gwadar,65.43,56,0,6.64,25.1216,62.3254,


In [13]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {"radius": 5000,
                  "types": "lodging",
                   "keyword": "Hotel",
                   "key": g_key}

        lat = row['Latitude']
        long = row['Longitude']
        city = row['City']
        params['location'] = f"{lat}, {long}"

        hotel_name = requests.get(base_url, params=params).json()
        results = hotel_name['results']
        
        print(f"First hotel in {city} is {results[0]['name']}.") 
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except IndexError:
        print("No hotel found.")

First hotel in Rafaela is Amérian Rafaela Hotel.
First hotel in Punta Arenas is Hotel Dreams Del Estrecho.
First hotel in Aden is Sheraton Gold Mohur Hotel.
First hotel in Palauig is Blue Spruce Beach Resort.
First hotel in Gwadar is Zaver Pearl Continental Hotel.
No hotel found.
First hotel in Cidreira is Hotel Castelo.
First hotel in Kapaa is The ISO.
First hotel in Cabo San Lucas is Waldorf Astoria Los Cabos Pedregal.
First hotel in Nava is Hotel Villaloreto.
First hotel in Cartagena del ChairÃ¡ is Hotel Los Corceles.
First hotel in Cape Town is Atlanticview Cape Town Boutique Hotel.
No hotel found.
First hotel in ValparaÃ­so is Hotel Boutique 17.
First hotel in Guerrero Negro is Malarrimo Hotel and Restaurant.
First hotel in Pecos is Best Western Plus Swiss Chalet Hotel & Suites.
First hotel in Ancud is Panamericana Hotel Ancud.
First hotel in Port Antonio is Trident Hotel.
First hotel in Hilo is Hilo Hawaiian Hotel.
First hotel in Bubaque is Le Mango Lodge.
First hotel in El Chimb

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
figure.add_layer(markers)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))